In [ ]:
import pandas as pd
import requests
from zipfile import ZipFile
import json

def download_and_extract_data() -> pd.DataFrame:
    pass

In [10]:
response_API = requests.get("https://api-football-standings.azharimm.site/leagues")
print(response_API.status_code)

data = response_API.text
#json.loads(data)
print(data)

200
<!doctype html>
<html data-adblockkey="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANDrp2lz7AOmADaN8tA50LsWcjLFyQFcb/P2Txc58oYOeILb3vBw7J6f4pamkAQVSQuqYsKx3YzdUHCvbVZvFUsCAwEAAQ==_uf6VCleIMvJQGPU7G57HgH6sgyLixFkmQKieRfxgcW5WoYN1pjsuHkjGz6B3Y1aNtRVMsZK9xLdhJmtT9SHqKw==" lang="en" style="background: #2B2B2B;">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">
    <link rel="preconnect" href="https://www.google.com" crossorigin>
</head>
<body>
<div id="target" style="opacity: 0"></div>
<script>window.park = "eyJ1dWlkIjoiYzFmZGFkMDItYWQwNC00ZTUwLWFkNTAtM2E0NDkwNmY2NDA5IiwicGFnZV90aW1lIjoxNzM5OTA3NDI1LCJwYWdlX3VybCI6Imh0dHBzOi8vYXBpLWZvb3RiYWxsLXN0YW5kaW5ncy5hemhhcmltbS5zaXRlL2xlYWd1ZXMiLCJwYWdlX21ldGhvZCI6IkdFVCIsInBhZ2VfcmVxdWVzdCI6e30sInBhZ2VfaGVhZGVycyI6e30sImhvc3QiOiJhcGktZm9vdGJhbGwtc3RhbmRpbmdz

In [8]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
from pathlib import Path
from src.paths import RAW_DATA_weather_DIR

def download_and_extract_weather_data(start_date, end_date):
    """
    Downloads hourly weather data from the Open-Meteo API and saves it as a CSV

    Args:
        start_date (str): Start date in "YYYY-MM-DD" format.
        end_date (str): End date in "YYYY-MM-DD" format. 

    Returns:
        pd.DateFrame: A DataFrame containing hourly weather data with temperature values.
    """

    # Setup the Open-Meteo API client
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": 52.52,
        "longitude": 13.41,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "precipitation", "visibility"],
        "timeformat": "unixtime",
        "timezone": "Europe/London" 
    }

    try:
        # Feth data from the Open-Meteo API
        responses = openmeteo.weather_api(url, params=params)[0] # assuming single location

        # Process metadata
        print(f"Coordinates: {responses.Latitude()}°N {responses.Longitude()}°E")
        print(f"Elevation {responses.Elevation()} m asl")
        print(f"Timezone {responses.Timezone()} {responses.TimezoneAbbreviation()}")
        print(f"Timezone difference to GMT+0 {responses.UtcOffsetSeconds()} seconds")
        
        # Extract hourly data
        hourly = responses.hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
        hourly_visibility = hourly.Variables(2).ValuesAsNumpy()

        hourly_data = {"date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc="True"),
            end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc="True"),
            freq = pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )}

        hourly_data["temperature_2m"] = hourly_temperature_2m
        hourly_data["precipitation"] = hourly_precipitation
        hourly_data["visibility"] = hourly_visibility

        # Convert to DataFrame and process timestamps
        hourly_dataframe  = pd.DataFrame(data=hourly_data)

        hourly_dataframe["date"] = pd.to_datetime(hourly_dataframe["date"]).dt.floor("h").dt.tz_localize(None)

        # Save to file
        file_path = RAW_DATA_weather_DIR/f"weather_data_{start_date}_to{end_date}.csv"
        hourly_dataframe.to_csv(file_path, index=False)
        print(f"Weather data to saved to {file_path}")

        return hourly_dataframe
    except Exception as e:
        print(f"Error downloading weather data : {e}")
        return pd.DataFrame()



ModuleNotFoundError: No module named 'openmeteo_requests'

In [7]:
weather_data = download_and_extract_weather_data("", "")
weather_data

NameError: name 'download_and_extract_weather_data' is not defined